# code for A/B tuning a RealSense D435

Code is very messy but what was used in the paper.

controls 
 - '1' -> prefer first
 - '2' -> prefer second
 - '3' -> heuristic denser
 - '4' -> heuristic sparser
 - 'q' -> quit the optimization by throwing exception. 
 
After the optimization is quit, another sorting round is done across all batches. Strictly we only need max but for debugging we wanted to see the top few vs the bottom few.

This will write a ~1GB log file with all the sequences and images in a pkl file that can be unpacked later. 

In [ ]:
import numpy as np
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib notebook
#%matplotlib qt

In [ ]:
#%matplotlib inline
import numpy as np


import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython import display
from ipywidgets import interact, widgets, interact_manual,interactive
import cv2

In [ ]:
# defaults
def_set = {
    'deepSeaMedianThreshold' : 500,
    'deepSeaNeighborThreshold' : 7,
    'deepSeaSecondPeakThreshold' : 325,
    'lrAgreeThreshold' : 24,
    'minusDecrement' : 10,
    'plusIncrement' : 10,
    'scoreThreshA' : 1,
    'scoreThreshB' : 2047,
    'textureCountThreshold' : 0,
    'textureDifferenceThreshold' : 0,
    'diffThresh' : 4,
    'removeThresh' : 63,
    'sloRauDiffThresh' : 1,
    'minEast' : 1,
    'minNorth' : 1,
    'minNSsum' : 3,
    'minSouth' : 1,
    'minWest' : 1,
    'minWEsum' : 3,
    'uShrink' : 3,
    'vShrink' : 1,
    'rauDiffThresholdBlue' : 51,
    'rauDiffThresholdGreen' : 51,
    'rauDiffThresholdRed' : 51,
    'diffThresholdBlue' : 72,
    'diffThresholdGreen' : 72,
    'diffThresholdRed' : 72,
    'sloK1Penalty' : 60,
    'sloK1PenaltyMod1' : 105,
    'sloK1PenaltyMod2' : 70,
    'sloK2Penalty' : 342,
    'sloK2PenaltyMod1' : 190,
    'sloK2PenaltyMod2' : 130,
    'lambdaAD' : 800,
    'lambdaCensus' : 26,
}

cols = sorted(def_set.keys())
use_f = np.array([False if c not in ['diffThresh','sloRauDiffThresh'] else True for c in cols])

In [ ]:
#
# SETUP
###############################################
import time
import pyrealsense2.pyrealsense2 as rs
cv2.destroyAllWindows()

# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

# Get device product line for setting a supporting resolution
pipeline_wrapper = rs.pipeline_wrapper(pipeline)
pipeline_profile = config.resolve(pipeline_wrapper)
device = pipeline_profile.get_device()
device_product_line = str(device.get_info(rs.camera_info.product_line))

found_rgb = False
for s in device.sensors:
    if s.get_info(rs.camera_info.name) == 'RGB Camera':
        found_rgb = True
        break
if not found_rgb:
    print("The demo requires Depth camera with Color sensor")
    exit(0)

config.enable_stream(rs.stream.depth, 848, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.infrared, 848, 480, rs.format.y8, 30)

#config.enable_stream(rs.stream.color, 1280, 720, rs.format.rgb8, 30)

# Start streaming
pipeline.start(config)
firstTime=True


query_min_values_mode = 1
query_max_values_mode = 2
advnc_mode = rs.rs400_advanced_mode(device)
print("Advanced mode is", "enabled" if advnc_mode.is_enabled() else "disabled")

# Loop until we successfully enable advanced mode
while not advnc_mode.is_enabled():
    print("Trying to enable advanced mode...")
    advnc_mode.toggle_advanced_mode(True)
    # At this point the device will disconnect and re-connect.
    print("Sleeping for 5 seconds...")
    time.sleep(5)
    # The 'dev' object will become invalid and we need to initialize it again
    advnc_mode = rs.rs400_advanced_mode(device)
    print("Advanced mode is", "enabled" if advnc_mode.is_enabled() else "disabled")


In [ ]:
get_func = [advnc_mode.get_depth_control,
 advnc_mode.get_rsm,
 advnc_mode.get_rau_support_vector_control,
 advnc_mode.get_rau_thresholds_control,
 advnc_mode.get_slo_color_thresholds_control,
 advnc_mode.get_slo_penalty_control,
 advnc_mode.get_hdad]
set_func = [advnc_mode.set_depth_control,
 advnc_mode.set_rsm,
 advnc_mode.set_rau_support_vector_control,
 advnc_mode.set_rau_thresholds_control,
 advnc_mode.set_slo_color_thresholds_control,
 advnc_mode.set_slo_penalty_control,
 advnc_mode.set_hdad]
min_set = {'lambdaCensus': 2, 'lambdaAD': 1}
max_set = {'lambdaCensus': 512, 'lambdaAD': 4096}
for func in get_func[:-1]:
    res = func(query_min_values_mode)
    names = [_ for _ in dir(res) if not _.startswith('__')]
    values = [getattr(res,n) for n in names]
    for n,v in zip(names,values):
        min_set[n] = v
for func in get_func[:-1]:
    res = func(query_max_values_mode)
    names = [_ for _ in dir(res) if not _.startswith('__')]
    values = [getattr(res,n) for n in names]
    for n,v in zip(names,values):
        max_set[n] = v
base_v = []
n2f = {}
for i,func in enumerate(get_func):
    res = func()
    base_v.append(res)
    names = [_ for _ in dir(res) if not _.startswith('__')]
    for n in names:
        n2f[n] = i

#def_set = {}
#for func in get_func:
#    res = func()
#    names = [_ for _ in dir(res) if not _.startswith('__')]
#    values = [getattr(res,n) for n in names]
#    for n,v in zip(names,values):
#        def_set[n] = v

In [ ]:
bounds = np.stack([np.array([min_set[k] for k in cols]), 
                      np.array([max_set[k] for k in cols])])
bounds = np.log(bounds+1)
cov_set = [def_set]
nrm_set = np.log(1+np.array([[s[k] for k in cols] for s in cov_set]))

# how many frames to buffer
BUF_F = 5

# how many frames to accumulate for tuning
FRAME_AVG =1 # BUF_F*4

CMAP = 'turbo'
VMIN = 150
VMAX = 1200

In [ ]:
depth_sensor = device.query_sensors()[0]
depth_sensor.set_option(rs.option.emitter_enabled, 0)
for i in range(3*BUF_F):
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    if not depth_frame:# or not color_frame:
        continue
import time
time.sleep(0.5)
for i in range(2*BUF_F):
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    left_frame = frames.get_infrared_frame()
    if not depth_frame or not left_frame:
        continue
c_frame = np.asanyarray(left_frame.get_data()).astype(np.float64)
depth_sensor = device.query_sensors()[0]
depth_sensor.set_option(rs.option.emitter_enabled, 0)
time.sleep(0.1)
for i in range(BUF_F):
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    left_frame = frames.get_infrared_frame()
    if not depth_frame or not left_frame:
        continue

In [ ]:
plt.imshow((c_frame/255.0)**(1/2.2),cmap='Greys_r')
plt.axis('off')
from PIL import Image
Image.fromarray(np.round(255*((c_frame/255.0)**(1.0/1.4))).astype(np.uint8) ).save("ref_img.png")

In [ ]:
TARGET_STD = 0.2
std_s = np.std(nrm_set,0)
print(std_s.mean())
if std_s.mean() < TARGET_STD:
    amt = TARGET_STD - std_s.mean()
    std_s += amt
    print(amt)
else:
    std_s /= std_s.mean()
    std_s += 0.1
std_s /= std_s.mean()

In [ ]:
import cma
es = cma.CMAEvolutionStrategy(nrm_set.mean(0),TARGET_STD,{'bounds':list(bounds)})

In [ ]:
# this will quit if 'q' is hit and throw an exception. That's fine. Continue with cells below

cv2.destroyAllWindows()
all_res = []
opt_iter = 0
best_subsets = [nrm_set.mean(0)]
while not es.stop():
    opt_iter += 1
    solutions = es.ask()
    N = len(solutions)
    vis_res = {}
    ind_r = []

    for i in range(N):
        og_v = solutions[i]
        og_v = np.clip(og_v,bounds[0],bounds[1])
        new_v = np.exp(og_v)-1
        new_v = np.where(use_f,new_v,new_v.round(0))
        try:
            for c,v in zip(cols,new_v):
                setattr(base_v[n2f[c]],c,int(v) if c not in ['diffThresh','sloRauDiffThresh'] else v)
            for j in range(len(set_func)):
                time.sleep(0.1)
                set_func[j](base_v[j])
                time.sleep(0.1)
            for j in range(BUF_F):
                frames = pipeline.wait_for_frames()
                depth_frame = frames.get_depth_frame()
            
            tot_img = np.asanyarray(depth_frame.get_data()).astype(np.float64)
            cnt_img = (tot_img != 0).astype(np.uint64)
            for k in range(FRAME_AVG):
                frames = pipeline.wait_for_frames()
                depth_frame = frames.get_depth_frame()
                depth_image = np.asanyarray(depth_frame.get_data()).astype(np.float64)

                depth_valid = (depth_image != 0).astype(np.uint64)
                cnt_img += depth_valid
                tot_img +=(depth_image)

                avg_img = tot_img/np.maximum(1,cnt_img)
            #except Exception as inst:
            #    print(i,inst)
            #    continue
            vis_res[i] = avg_img
        except Exception as inst:
            print(i,inst)
            continue
        #if (depth_image == 0).sum()/np.prod(depth_image.shape) > 0.3:
        #    continue
        vis_res[i] = avg_img
        ind_r.append((new_v,avg_img))

    import functools
    pair_comps = []

    def compare(i,j):
        cv2.destroyAllWindows()
        
        im1 =mpl.colormaps[CMAP](np.array(mpl.colors.Normalize(vmin=VMIN,vmax=VMAX,clip=True)(vis_res[i])))
        im2 =mpl.colormaps[CMAP](np.array(mpl.colors.Normalize(vmin=VMIN,vmax=VMAX,clip=True)(vis_res[j])))

        d1 = 100- 100*(vis_res[i] == 0).sum()/np.prod(vis_res[i].shape)
        d2 = 100- 100*(vis_res[j] == 0).sum()/np.prod(vis_res[j].shape)

        cv2.imshow("Comp {:.0f} {:.0f}".format(d1,d2), np.stack([im1,im2]).reshape((960,848,4)))

        while True:
            a = cv2.waitKey(30)
            if a == ord('1'):
                pair_comps.append([1,i,j])
                return 1
            if a == ord('2'):
                pair_comps.append([2,j,i])
                return -1
            if a == ord('3'):
                pair_comps.append([3] + ([i,j] if (d1 > d2) else [j,i]))
                return 2*(d1 >d2)-1
            if a == ord('4'):
                return 2*(d1 <d2)-1
                pair_comps.append([4] + ([i,j] if (d1 < d2) else [j,i]))
            if a == ord('q'):
                return
            else:
                continue
    def compare_density(i,j):
        d1 = 100- 100*(vis_res[i] == 0).sum()/np.prod(vis_res[i].shape)
        d2 = 100- 100*(vis_res[j] == 0).sum()/np.prod(vis_res[j].shape)
        return 2*(d1 >d2)-1
    #if opt_iter < 5:
    #    res = sorted(list(vis_res.keys()),key=functools.cmp_to_key(compare_density))
    #else:
    res = sorted(list(vis_res.keys()),key=functools.cmp_to_key(compare))
    
        
    batch_res = { 'params':[_[0] for _ in ind_r],
      'imgs': [_[1] for _ in ind_r],
      'order': res,
      'comps': pair_comps,
    }
    all_res.append(batch_res)
    
    
    best_subsets.append(solutions[res[-1]])
    t_vals = np.arange(len(solutions))/len(solutions)-opt_iter
    t_vals = t_vals[::-1] # rewards become costs
    r_vals = []
    for i in sorted(range(len(solutions))):
        if i in vis_res:
            v = t_vals[res.index(i)]
        else:
            v = t_vals.max()*1.1 # bad answer, give it bad
        r_vals.append(v)
    cv2.destroyAllWindows()
    es.tell(solutions, r_vals)
    es.logger.add()  # write data to disc to be plotted
    es.disp()

In [ ]:
vis_res = {}

In [ ]:
cv2.destroyAllWindows()
pair_comps = []
ind_r = []
for i in range(len(best_subsets)):
    if i in vis_res:
        continue
    time.sleep(0.25)
    og_v = best_subsets[i].copy()
    #og_v[24:27] = np.log(511+1)
    #og_v[27:30] = np.log(0+1)
    og_v = np.clip(og_v,bounds[0],bounds[1])
    new_v = np.exp(og_v)-1
    new_v = np.where(use_f,new_v,new_v.round(0))
    try:
        for k in range(BUF_F*2):
            frames = pipeline.wait_for_frames()
        for c,v in zip(cols,new_v):
            setattr(base_v[n2f[c]],c,int(v) if c not in ['diffThresh','sloRauDiffThresh'] else v)
        for j in range(len(set_func)):
            time.sleep(0.25)
            set_func[j](base_v[j])
            time.sleep(0.25)
            for k in range(BUF_F*2):
                frames = pipeline.wait_for_frames()
                depth_frame = frames.get_depth_frame()      

        tot_img = np.asanyarray(depth_frame.get_data()).astype(np.float64)
        cnt_img = (tot_img != 0).astype(np.uint64)
        for k in range(BUF_F*4):
            frames = pipeline.wait_for_frames()
            depth_frame = frames.get_depth_frame()
            depth_image = np.asanyarray(depth_frame.get_data()).astype(np.float64)

            depth_valid = (depth_image != 0).astype(np.uint64)
            cnt_img += depth_valid
            tot_img +=(depth_image)

            avg_img = tot_img/np.maximum(1,cnt_img)
            #except Exception as inst:
            #    print(i,inst)
            #    continue
            vis_res[i] = avg_img
        time.sleep(0.25)

    except Exception as inst:
        print(i,inst)
        continue
    #if (depth_image == 0).sum()/np.prod(depth_image.shape) > 0.3:
    #    continue
    vis_res[i] = avg_img
    ind_r.append((new_v,avg_img))


In [ ]:
res = sorted(list(vis_res.keys()),key=functools.cmp_to_key(compare))
cv2.destroyAllWindows()

In [ ]:
batch_res = { 'params':[_[0] for _ in ind_r],
  'imgs': [_[1] for _ in ind_r],
  'order': res,
  'comps': pair_comps,
}
all_res.append(batch_res)

In [ ]:
plt.figure()
for idx,i in enumerate(res):
    plt.subplot(4,4,idx+1)
    im1 =mpl.colormaps[CMAP](np.array(mpl.colors.Normalize(vmin=VMIN,vmax=VMAX,clip=True)(vis_res[i])))
    plt.imshow(im1)
    plt.axis('off')

In [ ]:
plt.figure()
im1 =mpl.colormaps[CMAP](np.array(mpl.colors.Normalize(vmin=VMIN,vmax=VMAX,clip=True)(vis_res[res[-1]])))

plt.imshow(im1)
plt.axis('off')

In [ ]:
Image.fromarray(((np.round(255*im1)).astype(np.uint8))).save("realsense_tune.png")

In [ ]:
og_v = np.clip(nrm_set.mean(0),bounds[0],bounds[1])
new_v = np.exp(og_v)-1
new_v = np.where(use_f,new_v,new_v.round(0))
for c,v in zip(cols,new_v):
    setattr(base_v[n2f[c]],c,int(v) if c not in ['diffThresh','sloRauDiffThresh'] else v)
for j in range(len(set_func)):
    time.sleep(0.1)
    set_func[j](base_v[j])
    time.sleep(0.1)
for j in range(BUF_F):
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()

tot_img = np.asanyarray(depth_frame.get_data()).astype(np.float64)
cnt_img = (tot_img != 0).astype(np.uint64)
for k in range(BUF_F*4):
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    depth_image = np.asanyarray(depth_frame.get_data()).astype(np.float64)

    depth_valid = (depth_image != 0).astype(np.uint64)
    cnt_img += depth_valid
    tot_img +=(depth_image)

    avg_img = tot_img/np.maximum(1,cnt_img)

im2 =mpl.colormaps[CMAP](np.array(mpl.colors.Normalize(vmin=VMIN,vmax=VMAX,clip=True)(avg_img)))

Image.fromarray(((np.round(255*im2)).astype(np.uint8))).save("default_night2.png")

In [ ]:
pipeline.stop()

In [ ]:
c_inits = [best_subsets[res[-1]]]
c_r = []
for c_i in c_inits:
    new_v = c_i.copy()
    #new_v[24:27] = np.log(511+1)

    new_v = np.clip(new_v,bounds[0],bounds[1])
    new_v = np.exp(new_v)-1
    new_v = np.where(use_f,new_v,new_v.round(0).astype(int))
    c_r.append({k:v for k,v in zip(cols,new_v)})

In [ ]:
results = {'cframe':c_frame,'cfgs':best_subsets,'res':res,'user_batches':all_res}
import glob
import pickle
N=len(glob.glob('man_results*.pkl'))
with open('man_results_{}.pkl'.format(N),'wb') as fp:
    pickle.dump(results,fp)

In [ ]:
out_res = {
    "device": {
        "fw version": "05.14.00.00",
        "name": "Intel RealSense D435",
        "product line": "D400"
    },
    "parameters": {
        "aux-param-autoexposure-setpoint": "1536",
        "aux-param-colorcorrection1": "0.298828",
        "aux-param-colorcorrection10": "-0",
        "aux-param-colorcorrection11": "-0",
        "aux-param-colorcorrection12": "-0",
        "aux-param-colorcorrection2": "0.293945",
        "aux-param-colorcorrection3": "0.293945",
        "aux-param-colorcorrection4": "0.114258",
        "aux-param-colorcorrection5": "-0",
        "aux-param-colorcorrection6": "-0",
        "aux-param-colorcorrection7": "-0",
        "aux-param-colorcorrection8": "-0",
        "aux-param-colorcorrection9": "-0",
        "aux-param-depthclampmax": "65536",
        "aux-param-depthclampmin": "0",
        "aux-param-disparityshift": "0",
        "controls-autoexposure-auto": "True",
        "controls-autoexposure-manual": "8500",
        "controls-color-autoexposure-auto": "True",
        "controls-color-autoexposure-manual": "166",
        "controls-color-backlight-compensation": "0",
        "controls-color-brightness": "0",
        "controls-color-contrast": "50",
        "controls-color-gain": "64",
        "controls-color-gamma": "300",
        "controls-color-hue": "0",
        "controls-color-power-line-frequency": "3",
        "controls-color-saturation": "64",
        "controls-color-sharpness": "50",
        "controls-color-white-balance-auto": "True",
        "controls-color-white-balance-manual": "4600",
        "controls-depth-gain": "16",
        "controls-laserpower": "150",
        "controls-laserstate": "on",
        "ignoreSAD": "0",
        "param-amplitude-factor": "0",
        "param-autoexposure-setpoint": "1536",
        "param-censusenablereg-udiameter": "9",
        "param-censusenablereg-vdiameter": "9",
        "param-censususize": "9",
        "param-censusvsize": "9",
        "param-depthclampmax": "65536",
        "param-depthclampmin": "0",
        "param-depthunits": "1000",
        "param-disableraucolor": "0",
        "param-disablesadcolor": "0",
        "param-disablesadnormalize": "0",
        "param-disablesloleftcolor": "0",
        "param-disableslorightcolor": "0",
        "param-disparitymode": "0",
        "param-disparityshift": "0",
        "param-lambdaad": "800",
        "param-lambdacensus": "26",
        "param-leftrightthreshold": "24",
        "param-maxscorethreshb": "2047",
        "param-medianthreshold": "500",
        "param-minscorethresha": "1",
        "param-neighborthresh": "7",
        "param-raumine": "1",
        "param-rauminn": "1",
        "param-rauminnssum": "3",
        "param-raumins": "1",
        "param-rauminw": "1",
        "param-rauminwesum": "3",
        "param-regioncolorthresholdb": "0.0499022",
        "param-regioncolorthresholdg": "0.0499022",
        "param-regioncolorthresholdr": "0.0499022",
        "param-regionshrinku": "3",
        "param-regionshrinkv": "1",
        "param-robbinsmonrodecrement": "10",
        "param-robbinsmonroincrement": "10",
        "param-rsmdiffthreshold": "4",
        "param-rsmrauslodiffthreshold": "1",
        "param-rsmremovethreshold": "0.375",
        "param-scanlineedgetaub": "72",
        "param-scanlineedgetaug": "72",
        "param-scanlineedgetaur": "72",
        "param-scanlinep1": "60",
        "param-scanlinep1onediscon": "105",
        "param-scanlinep1twodiscon": "70",
        "param-scanlinep2": "342",
        "param-scanlinep2onediscon": "190",
        "param-scanlinep2twodiscon": "130",
        "param-secondpeakdelta": "325",
        "param-texturecountthresh": "0",
        "param-texturedifferencethresh": "0",
        "param-usersm": "1",
        "param-zunits": "1000"
    },
    "schema version": 1,
    "viewer": {
        "stream-depth-format": "Z16",
        "stream-fps": "30",
        "stream-height": "480",
        "stream-width": "848"
    }
}
out_map = {'deepSeaMedianThreshold': 'medianthreshold',
 'deepSeaNeighborThreshold': 'neighborthresh',
 'deepSeaSecondPeakThreshold': 'secondpeakdelta',
 'diffThresh': 'rsmdiffthreshold',
 'diffThresholdBlue': 'scanlineedgetaub',
 'diffThresholdGreen': 'scanlineedgetaug',
 'diffThresholdRed': 'scanlineedgetaur',
 'lambdaAD': 'lambdaad',
 'lambdaCensus': 'lambdacensus',
 'lrAgreeThreshold': 'leftrightthreshold',
 'minEast':'raumine',
 'minNSsum': 'rauminnssum',
 'minNorth': 'rauminn',
 'minSouth': 'raumins',
 'minWEsum': 'rauminwesum',
 'minWest': 'rauminw',
 'minusDecrement': 'robbinsmonrodecrement',
 'plusIncrement': 'robbinsmonroincrement',
 'rauDiffThresholdBlue': 'regioncolorthresholdb',
 'rauDiffThresholdGreen': 'regioncolorthresholdg',
 'rauDiffThresholdRed': 'regioncolorthresholdr',
 'removeThresh': 'rsmremovethreshold',
 'scoreThreshA': 'minscorethresha',
 'scoreThreshB': 'maxscorethreshb',
 'sloK1Penalty': 'scanlinep1',
 'sloK1PenaltyMod1': 'scanlinep1onediscon',
 'sloK1PenaltyMod2': 'scanlinep2twodiscon',
 'sloK2Penalty': 'scanlinep2',
 'sloK2PenaltyMod1': 'scanlinep2onediscon',
 'sloK2PenaltyMod2': 'scanlinep2twodiscon',
 'sloRauDiffThresh': 'rsmrauslodiffthreshold',
 'textureCountThreshold': 'texturecountthresh',
 'textureDifferenceThreshold': 'texturedifferencethresh',
 'uShrink': 'regionshrinku',
 'vShrink': 'regionshrinkv'}
for k,v in c_r[0].items():
    if 'regioncolorthreshold' in out_map[k]:
        v = v/1022
    if 'rsmremovethreshold' in out_map[k]:
        v = v/168
    out_res['parameters']['param-' + out_map[k]] = str(int(v) if int(v) == v else round(v,7))

In [ ]:
name_base = 'MAN_TUNE_{}'.format(N)
import json
with open('{}.json'.format(name_base),'wt') as fp:
    json.dump(out_res,fp, sort_keys=True, indent=4)